Initalization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

Loading Table

In [0]:
df_silver = spark.table("workspace.bronze.erp_px_cat_g1v2")
df_silver.limit(10).display()

Exploring Data

In [0]:
df_silver.printSchema()
df_silver.summary().show()

In [0]:
df_silver.select("CAT").distinct().display()
df_silver.select("SUBCAT").distinct().display()
df_silver.select("MAINTENANCE").distinct().display()


Trimming Data

In [0]:
for field in df_silver.schema.fields:
    if isinstance(field.dataType, StringType):
        df_silver = df_silver.withColumn(field.name, trim(col(field.name)))

Cleaning Data

In [0]:

df_silver = df_silver.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)

Renaming Columns

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df_silver = df_silver.withColumnRenamed(old_name, new_name)

In [0]:
df_silver.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_product_category")